Aqui eu pego a VGG pré-treinada

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import Model

model = VGG16(weights="imagenet")
input_layer = model.input
selected_layer = model.get_layer(index = -5).output
flat_out = layers.Flatten()(selected_layer)

print(input_layer, selected_layer)
model = Model(inputs = input_layer, outputs = flat_out)
model.summary()

Tensor("input_1:0", shape=(None, 224, 224, 3), dtype=float32) Tensor("block5_pool/Identity:0", shape=(None, 7, 7, 512), dtype=float32)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147

Configuro o tf pra não ter nenhum estouro de memória

In [2]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
try:
	tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
	pass

In [ ]:
Tiro 100 da classe 0 e 100 da classe 1, tanto no treino quanto no teste

In [3]:
from tensorflow.keras import datasets
import numpy as np

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

def make_binary(x, y):
    neg = np.where(y == 0)
    pos = np.where(y == 1)
    neg = neg[0][:100]
    pos = pos[0][:100]
    print(len(pos), len(neg))
    index = np.concatenate((pos, neg), axis = -1)
    x = x[index]
    y = y[index]
    return x, y

train_images, train_labels = make_binary(train_images, train_labels)
test_images, test_labels = make_binary(test_images, test_labels)
# print(train_labels, len(train_labels))
# print(test_labels, len(test_labels))
# print(train_images.shape, test_images.shape)

100 100
100 100


In [4]:

train_images = tf.image.resize(train_images, [224,224])
test_images = tf.image.resize(test_images, [224,224])

print(train_images.shape, test_images.shape)
train_features = model.predict(x = train_images)
test_features = model.predict(x = test_images)

(200, 224, 224, 3) (200, 224, 224, 3)


In [5]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X = train_features, y = train_labels)

<ipython-input-5-6d270b5d2e66>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X = train_features, y = train_labels)


RandomForestClassifier()

In [6]:
def metrics(prediction, true_pred, nb_samples):
	from sklearn.metrics import accuracy_score
	from sklearn.metrics import precision_score
	from sklearn.metrics import recall_score
	from sklearn.metrics import f1_score
	from sklearn.metrics import confusion_matrix

	accuracy = accuracy_score(true_pred, prediction)
	precision = precision_score(true_pred, prediction)
	recall = recall_score(true_pred, prediction)
	f1 = f1_score(true_pred, prediction)
	tn, fp, fn, tp = confusion_matrix(true_pred, prediction).ravel()
	specificity = tn / (tn+fp)

	print('Accuracy: %f' % accuracy)
	print('Precision: %f' % precision)
	print('Recall: %f' % recall)
	print('F1 score: %f' % f1)
	print("Specificity = %f" % specificity)
    
predictions = clf.predict(test_features)
metrics(predictions, test_labels, len(predictions))

Accuracy: 0.885000
Precision: 0.859813
Recall: 0.920000
F1 score: 0.888889
Specificity = 0.850000
